In [1]:
import cadquery as cq

from jupyter_cadquery import (
    versions,
    show, PartGroup, Part, 
    get_viewer, close_viewer, get_viewers, close_viewers, open_viewer, set_defaults, get_defaults, open_viewer,
    get_pick,
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay
''
enable_replay(False)


import sys
sys.path.append('../')
from cadquery.occ_impl.geom import Vector
sys.path.append('../examples/Bens_alp')
from config import Config, Shape
from config import read_config_from_json

from plate import get_base, get_screw_positions, get_key_positions, make_plate
from case import get_basic_shape,  get_distance_between_two_shapes,  add_microcontrollerbox, make_controller_box_top_plate, CustomisableSelector

/home/ben/anaconda3/envs/jcq3/lib/python3.10/site-packages/quaternion/calculus.py:310: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def fd_indefinite_integral(f, t):


Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay


In [2]:
config_dict = """{
    "row_key_numbers": [
        7,
        10,
        10,
        10
    ],
    "columnSpacing": 19,
    "rowSpacing": 19,
    "plateThickness": 2,
    "screwHoleDiamater": 2.4,
    "switchHoleSize": 13.9,
    "shape": 0,
    "notched_keyhole": true,
    "caseHeight": 22,
    "caseGap": 1.0,
    "wallThickness": 1.6,
    "floorThickness": 3.0,
    "edgeFillet": 3,
    "bottomFillet": 2,
    "controller": {
        "name": "Pi Pico",
        "board_dimension_x": 51.0,
        "board_dimension_y": 21.0,
        "screw_hole_x": 47.0,
        "screw_hole_y": 11.4
    },
    "controllerYOffset": 6,
    "controllerBoxThickness": 2
}"""

config = read_config_from_json(string=config_dict)

config.controller

ControllerFromDict(dict={'name': 'Pi Pico', 'board_dimension_x': 51.0, 'board_dimension_y': 21.0, 'screw_hole_x': 47.0, 'screw_hole_y': 11.4}, board_dimension_x=51.0, board_dimension_y=21.0, screw_hole_x=47.0, screw_hole_y=11.4)

In [6]:
def make_case(config:Config, 
              modify_controller_box:callable = None,
              get_screw_positions:callable = get_screw_positions,
              cut_hole_for_connector:callable = None,
              ) -> cq.Sketch:
   
    
    case= get_basic_shape(config)
    case.edges('>Z').tag('outerTopEdge') # for making controller box top plate

    # scoop out interior
    case= (case.faces(">Z or <Z").shell(config.wallThickness, kind='intersection'))
    
    # set floor thickness 
    case = case.faces('<Z').wires('>X and >Y').toPending().extrude(config.floorThickness)
    
    #fillet bottom edge
    if config.bottomFillet != 0:
        case = case.edges('<Z').fillet(config.bottomFillet)


    

    # add controllerbox holes.
    if config.controller is not None:
        hole_x_pos = config.controller.screw_hole_x
        hole_y_pos = config.controller.screw_hole_y- config.wallThickness - config.controllerYOffset - (config.controller.board_dimension_y -config.controller.screw_hole_y)*0.5
        case = (case.workplaneFromTagged("controllerBox")
                .rect(hole_x_pos, hole_y_pos, forConstruction=True, centered=[True,True,True])
                .vertices()
                .cskHole(config.screwHoleDiamater, 2*config.screwHoleDiamater, 82, depth=None)
        )
    
  

    return case



case = make_case(config,)

case = (case.faces(tag='controllerBox').workplane(centerOption="CenterOfBoundBox", invert=False)
                .center(0, config.controllerYOffset )
                .rect(config.controller.screw_hole_x, config.controller.screw_hole_y, forConstruction=True, centered=[True,True,True])
                #.extrude(20, combine=True)
                 .vertices()
                # ''.box(5,5,5)
                .cskHole(config.screwHoleDiamater, 2*config.screwHoleDiamater, 82, depth=None)
        )


In [7]:
get_base

CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='light', title=None,…

In [41]:
cq.exporters.export(case, 'pico_test.stl')